In [297]:
# imports
import numpy as np
import random
import os

In [298]:
def espaco(linhas):
  return np.zeros((linhas, linhas))

In [299]:
def pega_individuo_nulo(matrix):
    shape = np.shape(matrix)

    x_aleatorio = random.randint(0, shape[0]-1)
    y_aleatorio = random.randint(0, shape[0]-1)

    individuo = matrix[x_aleatorio][y_aleatorio]

    if individuo == 0:
        matrix[x_aleatorio][y_aleatorio] = 1
    else:
        pega_individuo_nulo(matrix)

    return matrix


In [300]:
def individuos(matrix, num_individuos):
    for i in range(0, num_individuos):
        pega_individuo_nulo(matrix)
    return matrix

In [301]:
def vizinhanca(matriz):
    linhas, colunas = matriz.shape
    max_vizinhos = 8 

    populacao = np.zeros((linhas * colunas, 3 + max_vizinhos), dtype=int)

    k = 0
    for i in range(linhas):
        for j in range(colunas):
            populacao[k, 1] = i
            populacao[k, 2] = j

            vizinhos = []
            for x in range(i - 1, i + 2):
                for y in range(j - 1, j + 2):
                    if 0 <= x < linhas and 0 <= y < colunas and (x != i or y != j):
                        vizinhos.append((x, y))

            # Ajuste para garantir que não ultrapasse o número máximo de vizinhos
            vizinhos = vizinhos[:max_vizinhos]

            # Preenchendo os vizinhos na matriz populacao
            populacao[k, 3:(3 + len(vizinhos))] = [x * colunas + y for x, y in vizinhos]
            k += 1

    return populacao, max_vizinhos

In [302]:
def infectar(populacao, numero):
    indices_suscetiveis = np.where(populacao[:, 0] == 0)[0]
    indices_infectar = np.random.choice(indices_suscetiveis, size=numero, replace=False)
    populacao[indices_infectar, 0] = 1
    return populacao

In [303]:
def ruido(populacao, numero, prob):
    indices_ruido = np.random.choice(populacao.shape[0], size=numero, replace=False)
    for i in indices_ruido:
        if random.random() < prob and populacao[i, 0] == 1:
            populacao[i, 0] = 1 - populacao[i, 0]  # Inverte o estado de infecção
    return populacao

In [304]:
def atualizar(populacao, numero, prob):
    nova_matriz = populacao.copy()
    for i in range(populacao.shape[0]):
        if populacao[i, 0] == 0:  # Apenas para indivíduos suscetíveis

            vizinhos_infectados = np.sum(populacao[populacao[i, 3:11], 0] == 1)
            if vizinhos_infectados >= numero and np.random.rand() < prob:
                nova_matriz[i, 0] = 1  # Indivíduo suscetível foi infectado
    return nova_matriz


In [305]:
def gravar(populacao, diretorio, nome_arquivo):
    nome_arquivo = os.path.join(diretorio, nome_arquivo)
    with open(nome_arquivo, 'w') as arquivo:
        arquivo.write("x y estado\n")
        for linha in (populacao):
            x, y = int(linha[1]), int(linha[2])
            estado = int(linha[0])
            arquivo.write(f"{x} {y} {estado}\n")

In [306]:
def vacinacao(populacao, num_vacinacao):
    vacinados = 0
    nova_matriz = populacao.copy()
    while vacinados <= num_vacinacao:
        lista_suscetiveis = np.where(nova_matriz[:, 0] == 0)[0]
        if len(lista_suscetiveis) > 0:
            sortudo = np.random.choice(lista_suscetiveis)
            nova_matriz[sortudo, 0] = 2
            vacinados = vacinados+1
            # O suscetível será removido da populacao quando seu estado for alterado
        else:
            print("Não existem indivíduos suscetíveis para serem vacinados")
            break
    return nova_matriz

In [307]:
tamanho_matriz = 10
num_inicial_de_individuos = 5
num_aleatorio_de_infectados_iniciais = 5

num_min_de_vizinhos_infectados = 2
chance_de_infeccao_com_vizinho = 0.05

num_individuos_de_ruido = 10
chance_de_infeccao_de_ruido = 0.005

matrix = espaco(tamanho_matriz)
matrix_com_individuos = individuos(matrix, num_inicial_de_individuos)
populacao_info, max_vizinhos = vizinhanca(matrix_com_individuos)
populacao_info = infectar(populacao_info, num_aleatorio_de_infectados_iniciais)


n_epocas = 300
epocas_vacinacao = [50, 100, 150]
num_vacinacao = 33

for tempo in range(1, n_epocas):
    populacao_info = atualizar(populacao_info, num_min_de_vizinhos_infectados, chance_de_infeccao_com_vizinho)
    populacao_info = ruido(populacao_info, num_individuos_de_ruido, chance_de_infeccao_de_ruido)
    if tempo in epocas_vacinacao:
        populacao_info = vacinacao(populacao_info, num_vacinacao)
    
    gravar(populacao_info, "epocas", f"estado_{tempo}.txt")


Não existem indivíduos suscetíveis para serem vacinados
Não existem indivíduos suscetíveis para serem vacinados


In [308]:
def ler_populacao():
    diretorio = "epocas"
    arquivos = [arquivo for arquivo in os.listdir(diretorio) if arquivo.endswith(".txt")]
    arquivos.sort(key=lambda x: int(x.split("_")[1].split(".")[0]))

    dados_populacao = []

    for arquivo in arquivos:
        caminho_arquivo = os.path.join(diretorio, arquivo)
        with open(caminho_arquivo, 'r') as file:
            # Ignorando a primeira linha (cabeçalho)
            linhas = file.readlines()[1:]
            dados = [linha.strip().split() for linha in linhas]
            dados = np.array([[int(x), int(y), int(estado)] for x, y, estado in dados])
            dados_populacao.append(dados)

    return dados_populacao

In [309]:
import imageio
from io import BytesIO
from matplotlib import pyplot as plt

In [310]:
def criar_matriz_visual_bytes(dados):
    coordenadas_x = [x for x, _, _ in dados]
    coordenadas_y = [y for _, y, _ in dados]
    estados = [estado for _, _, estado in dados]

    fig, ax = plt.subplots()
    scatter = ax.scatter(coordenadas_x, coordenadas_y, c=estados, cmap='gray_r', s=100, marker='s')
    ax.set_xticks(range(max(coordenadas_x) + 1))
    ax.set_yticks(range(max(coordenadas_y) + 1))
    ax.grid(True)

    # Converter a figura para bytes
    buf = BytesIO()
    plt.savefig(buf, format='png')
    plt.close(fig)


    return buf.getvalue()

In [311]:
def excluir_arquivos_na_pasta(caminho_da_pasta = "epocas"):
    # Obtém a lista de arquivos na pasta
    arquivos = os.listdir(caminho_da_pasta)

    # Itera sobre a lista de arquivos e os exclui
    for arquivo in arquivos:
        caminho_completo = os.path.join(caminho_da_pasta, arquivo)
        if os.path.isfile(caminho_completo):
            os.remove(caminho_completo)

In [312]:
populacao = ler_populacao()
lista_figuras = []

# Lista para armazenar as figuras em bytes
lista_figuras_bytes = []

# Criar as figuras
for dados_iteracao in populacao:
    figura_bytes = criar_matriz_visual_bytes(dados_iteracao)
    lista_figuras_bytes.append(figura_bytes)

# Criar o arquivo GIF
with imageio.get_writer('matriz_visual.gif', duration=0.5) as writer:
    for figura_bytes in lista_figuras_bytes:
        imagem = imageio.imread(figura_bytes, format='png')
        writer.append_data(imagem)



C:\Users\Filipe\AppData\Local\Temp\ipykernel_8508\2106218463.py:15: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  imagem = imageio.imread(figura_bytes, format='png')


In [313]:
excluir_arquivos_na_pasta()